In [3]:
from create_book import (
    retrieve_story,
    set_cover,
    set_metadata,
    add_chapters,
    slugify,
    wp_get_cookies,
    fetch_part_content
)

In [24]:
cookies = await wp_get_cookies(username="", password="") 

ValueError: No cookies.

In [3]:
cookies

{'usprivacy': '',
 'lang': '34',
 'locale': 'en_US',
 'wp_id': 'c890f1ce-3888-4f63-b75d-becfd6e56d41',
 'token': '369518880%3A2%3A1719074105%3AgdSoKCmLBvVs71tpGeJPGETmGmDIdJYmltCbPeM6Awt990JYBUHqSPdOpMO1OVRD'}

In [6]:
d = await retrieve_story(369237858, cookies=cookies)

In [7]:
d

{'id': '369237858',
 'title': "Of Rust and Gold [The Saints' Song Book 1]",
 'createDate': '2024-05-18T18:59:36Z',
 'modifyDate': '2024-05-22T15:16:33Z',
 'language': {'name': 'English'},
 'user': {'username': 'Poindexter'},
 'description': "On the hunt for a cannibalistic living god, the last thing a politically ambitious prince has time for is the foul-mouthed pirate he can't stop thinking about.\n\n                                              *****\n\nAs ninth in line to the throne--and a legitimized royal bastard to boot--Prince Argus is desperate for the political power that will make the world see him as an equal to his brothers. A favor for a favor should get him everything he wants, but finding another prince's stolen ship and its stolen cargo means befriending his only lead on the theft: a captured pirate named Leonardo whose gruff and foul-mouthed exterior hides more than Argus bargained for. Because as illicit feelings begin to grow between them, Argus faces a harsh reality

In [8]:
d['parts'][0]['id']

1446920685

In [14]:
cookies

{'usprivacy': '',
 'lang': '34',
 'locale': 'en_US',
 'wp_id': 'c890f1ce-3888-4f63-b75d-becfd6e56d41',
 'token': '369518880%3A2%3A1719074105%3AgdSoKCmLBvVs71tpGeJPGETmGmDIdJYmltCbPeM6Awt990JYBUHqSPdOpMO1OVRD'}

In [13]:
await fetch_part_content(1446923091, cookies=cookies)

'<p data-p-id="fb195658a08a48e18c55c8e041cf07bd">This story is part of the Paid Stories program, a selection of exclusive stories that help support Wattpad writers. <a href="https://my.w.tt/1CugQ53VB7">Click here</a> to continue reading on the Wattpad app.<br></p>\n<p data-p-id="fb195658a08a48e18c55c8e041cf07bd">Esta historia pertenece al programa de Historias Pagadas, una selección exclusiva de historias con las que apoyas a escritores de Wattpad. <a href="https://my.w.tt/1CugQ53VB7">Da click aquí</a> para seguir leyendo en la aplicación.<br><br></p>\n<p data-media-type="image" data-image-layout="one-horizontal" data-p-id="c525e05b7f45fbf2a48971162963f17c"><img style="background-color: initial;" data-original-width="706" data-original-height="192" src="https://s3.amazonaws.com/in-app-purchase-content-public/wordmark.png"></p>\n'

In [12]:
from aiohttp import ClientSession

In [16]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
}

In [19]:
async with ClientSession(headers=headers, cookies=cookies) as session:
    async with session.get("https://www.wattpad.com/apiv2/?m=storytext&id=1446923091") as response:
        print(await response.text())

<p data-p-id="faa20ecdcbd7df748793c033cd8d1db3"><b>10. If The Sea Could Talk</b></p><p data-p-id="a2850cad7d5beb26e0d9d11b9f736a64">Leo was getting tired of running while fear scorched his lungs. He didn't stop until the brothel was far behind him, until Mel's dragging weight made him stumble and Mung stopped ahead of them to brace the side of a building and catch his breath. He wiped sweaty clumps of hair from his eyes. "Gods forgive me."</p><p data-p-id="2e33bd54ce316ad475d9b16eeec00725">Leo threw Mel in front of him. She gripped on tightly to her small sword, still holding it out as if she didn't want it to touch her. The metal was stained red with tiny droplets. </p><p data-p-id="a4eb12ef4409e123308fddb1d8e278e4">"Why did you cut his ear off!" </p><p data-p-id="d2c898bea78b70ff2718bf811c0499d2">She flinched. </p><p data-p-id="de2a63d35bbbafe783327287606a590b">They would have been alright, better off at least, if they'd let Engli scream his head off trying to remove his ax from the 

In [17]:
from ebooklib import epub
story_id = 369241212

In [15]:
import asyncio
from ebooklib import epub
import unicodedata
import re
from aiohttp_client_cache import FileBackend

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
}

cache = FileBackend(
    use_temp=True,
    expire_after=43200,  # 12 hours
)


async def wp_get_cookies(username: str, password: str) -> dict:
    """Retrieves authorization cookies from Wattpad by logging in with user creds.

    Args:
        username (str): Username.
        password (str): Password.

    Raises:
        ValueError: Bad status code.
        ValueError: No cookies returned.

    Returns:
        dict: Authorization cookies.
    """
    async with ClientSession(headers=headers) as session:
        async with session.post(
            "https://www.wattpad.com/auth/login?nextUrl=%2F&_data=routes%2Fauth%2Flogin",
            data={
                "username": username.lower(),
                "password": password,
            },  # the username.lower() is for caching
        ) as response:
            if response.status != 204:
                raise ValueError("Not a 204.")
            print(response.cookies, await response.text())
            cookies = {
                k: v.value
                for k, v in response.cookies.items()  # Thanks https://stackoverflow.com/a/32281245
            }

            if not cookies:
                raise ValueError("No cookies.")

            return cookies


async def retrieve_story(story_id: int, retry=True, cookies={}) -> dict:
    """Taking a story_id, return its information from the Wattpad API."""
    async with ClientSession(
        headers=headers,
        cookies=cookies,
    ) as session:
        try:
            async with session.get(
                f"https://www.wattpad.com/api/v3/stories/{story_id}?fields=tags,id,title,createDate,modifyDate,language(name),description,completed,mature,url,isPaywalled,user(username),parts(id,title),cover"
            ) as response:
                if not response.ok:
                    if response.status in [404, 400]:
                        return {}
                    raise ValueError("Status Code:", response.status)
                body = await response.json()
        except ValueError:
            if not retry:
                raise asyncio.TimeoutError()
            await asyncio.sleep(15)
            return await retrieve_story(story_id, retry=False)

    return body


async def fetch_part_content(part_id: int, retry: bool = True, cookies={}) -> str:
    """Return the HTML Content of a Part."""
    print('fc cookies', cookies)
    async with ClientSession(headers=headers, cookies=cookies) as session:
        async with session.get(
            f"https://www.wattpad.com/apiv2/?m=storytext&id={part_id}"
        ) as response:
            if not response.ok:
                if response.status in [404, 400]:
                    return ""
                raise ValueError("Status Code:", response.status)
            body = await response.text()

    return body


async def fetch_cover(url: str, cookies={}) -> bytes:
    """Fetch image bytes."""
    async with ClientSession(headers=headers, cookies=cookies) as session:
        async with session.get(url) as response:
            if not response.ok:
                if response.status in [404, 400]:
                    return bytes()
                raise ValueError("Status Code:", response.status)
            body = await response.read()

    return body


def slugify(value, allow_unicode=False) -> str:
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.

    Thanks https://stackoverflow.com/a/295466.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize("NFKC", value)
    else:
        value = (
            unicodedata.normalize("NFKD", value)
            .encode("ascii", "ignore")
            .decode("ascii")
        )
    value = re.sub(r"[^\w\s-]", "", value.lower())
    return re.sub(r"[-\s]+", "-", value).strip("-_")


# --- #


def set_metadata(book, data):
    book.add_author(data["user"]["username"])

    book.add_metadata("DC", "description", data["description"])
    book.add_metadata("DC", "created", data["createDate"])
    book.add_metadata("DC", "modified", data["modifyDate"])
    book.add_metadata("DC", "language", data["language"]["name"])

    book.add_metadata(
        None, "meta", "", {"name": "tags", "content": ", ".join(data["tags"])}
    )
    book.add_metadata(
        None, "meta", "", {"name": "mature", "content": str(int(data["mature"]))}
    )
    book.add_metadata(
        None, "meta", "", {"name": "completed", "content": str(int(data["completed"]))}
    )


async def set_cover(book, data):
    book.set_cover("cover.jpg", await fetch_cover(data["cover"]))


async def add_chapters(book, data, cookies={}):
    chapters = []
    print('cookies', cookies)

    for part in data["parts"]:
        content = await fetch_part_content(part["id"], cookies=cookies)
        title = part["title"]

        # Thanks https://eu17.proxysite.com/process.php?d=5VyWYcoQl%2BVF0BYOuOavtvjOloFUZz2BJ%2Fepiusk6Nz7PV%2B9i8rs7cFviGftrBNll%2B0a3qO7UiDkTt4qwCa0fDES&b=1
        chapter = epub.EpubHtml(
            title=title,
            file_name=f"{slugify(title)}.xhtml",
            lang=data["language"]["name"],
        )
        chapter.set_content(f"<h1>{title}</h1>" + content)

        chapters.append(chapter)

        yield title  # Yield the chapter's title upon insertion preceeded by retrieval.

    for chapter in chapters:
        book.add_item(chapter)

    book.toc = tuple(chapters)

    # Thanks https://github.com/aerkalov/ebooklib/blob/master/samples/09_create_image/create.py
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # create spine
    book.spine = ["nav"] + chapters


In [18]:
import os

data = await retrieve_story(story_id, cookies=cookies)
book = epub.EpubBook()

# Metadata and Cover are updated
set_metadata(book, data)
await set_cover(book, data)
print("Metadata Downloaded")

# Chapters are downloaded
async for title in add_chapters(book, data, cookies=cookies):
    # print(f"Part ({title}) downloaded")
    ...

epub.write_epub(f'{slugify(data["title"])}_{story_id}.epub', book, {})


Metadata Downloaded
cookies {'usprivacy': '', 'lang': '34', 'locale': 'en_US', 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608', 'token': '369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH'}
fc cookies {'usprivacy': '', 'lang': '34', 'locale': 'en_US', 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608', 'token': '369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH'}
fc cookies {'usprivacy': '', 'lang': '34', 'locale': 'en_US', 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608', 'token': '369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH'}
fc cookies {'usprivacy': '', 'lang': '34', 'locale': 'en_US', 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608', 'token': '369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH'}
fc cookies {'usprivacy': '', 'lang': '34', 'locale': 'en_US', 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608', 'tok

In [19]:
cookies

{'usprivacy': '',
 'lang': '34',
 'locale': 'en_US',
 'wp_id': 'f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608',
 'token': '369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH'}

In [23]:
async with CachedSession(cache=cache, cookies=cookies) as session:
    async with session.get("https://httpbin.org/cookies") as response:
        print(awaitresponse.text())

{
  "cookies": {
    "lang": "34", 
    "locale": "en_US", 
    "token": "369518880%3A2%3A1719075107%3AcbCP3ZM2Xop4cdg3oD194g3PnZ6gJ_IPu3kKnMMm7iyEeghAUSdumpQ8mfhMRNoH", 
    "usprivacy": "", 
    "wp_id": "f2bbf3eb-7bf0-4ee0-8ed8-cd1ee02b2608"
  }
}

